## Solutions - Problem 4

Check if there are any origins in airtraffic data which do not have correpsonding records in airport-codes.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/ftYeFELu5D4?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

* This is an example for outer join.
* We need to get those airports which are in Origin field in January 2008 airtraffic data set but not in airport-codes. We need to consider all the valid records from airport codes.
* Based on the side of the airtraffic data set, we can say left or right. We will be invoking join using airtraffic data set and hence we will use left outer join.
* We will also apply distinct on Origin before performing left outer join.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Joining Data Sets'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [3]:
airtrafficPath = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [4]:
airtraffic = spark. \
    read. \
    parquet(airtrafficPath)

In [5]:
airtraffic. \
    select(
        "Year", "Month", "DayOfMonth", 
        "Origin", "Dest", "CRSDepTime"
    ). \
    show()

+----+-----+----------+------+----+----------+
|Year|Month|DayOfMonth|Origin|Dest|CRSDepTime|
+----+-----+----------+------+----+----------+
|2008|    1|        16|   BGR| CVG|      1735|
|2008|    1|        17|   SYR| CVG|      1701|
|2008|    1|        17|   SAV| BOS|      1225|
|2008|    1|        17|   CVG| GRR|      1530|
|2008|    1|        17|   STL| CVG|      1205|
|2008|    1|        18|   STL| JFK|      1150|
|2008|    1|        18|   MCI| CVG|      1009|
|2008|    1|        19|   TUL| CVG|       835|
|2008|    1|        20|   JFK| PHL|      1935|
|2008|    1|        20|   RDU| CVG|       830|
|2008|    1|        21|   CVG| DTW|      1640|
|2008|    1|        21|   MSY| LGA|      1204|
|2008|    1|        21|   JFK| PHL|      1935|
|2008|    1|        21|   DCA| JFK|      1830|
|2008|    1|        21|   HSV| DCA|       700|
|2008|    1|        22|   ORD| CVG|      1910|
|2008|    1|        22|   CVG| JFK|      1320|
|2008|    1|        23|   LGA| SAV|       908|
|2008|    1| 

In [6]:
airtraffic.count()

605659

In [7]:
airportCodesPath = "/public/airtraffic_all/airport-codes"

In [8]:
def getValidAirportCodes(airportCodesPath):
    airportCodes = spark. \
        read. \
        option("sep", "\t"). \
        option("header", True). \
        option("inferSchema", True). \
        csv(airportCodesPath). \
        filter("!(State = 'Hawaii' AND IATA = 'Big')")
    return airportCodes

In [9]:
airportCodes = getValidAirportCodes(airportCodesPath)

In [10]:
airportCodes.count()

524

In [11]:
airtraffic. \
    select("Origin"). \
    distinct(). \
    show()

+------+
|Origin|
+------+
|   BGR|
|   SYR|
|   CVG|
|   STL|
|   JFK|
|   MSY|
|   DCA|
|   HSV|
|   ORD|
|   CLT|
|   GSP|
|   BOS|
|   COS|
|   BNA|
|   ATL|
|   SJC|
|   GJT|
|   AZO|
|   ELP|
|   PDX|
+------+
only showing top 20 rows



In [15]:
airtraffic. \
    select("Origin"). \
    distinct(). \
    count()

286

In [12]:
airtraffic. \
    select("Origin"). \
    distinct(). \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA"], "left"). \
    show()

+------+----------------+-----+-------+----+
|Origin|            City|State|Country|IATA|
+------+----------------+-----+-------+----+
|   BGR|          Bangor|   ME|    USA| BGR|
|   SYR|        Syracuse|   NY|    USA| SYR|
|   CVG|      Cincinnati|   OH|    USA| CVG|
|   STL|       St. Louis|   MO|    USA| STL|
|   JFK|        New York|   NY|    USA| JFK|
|   MSY|     New Orleans|   LA|    USA| MSY|
|   DCA|   Washington DC| null|    USA| DCA|
|   HSV|      Huntsville|   AL|    USA| HSV|
|   ORD|         Chicago|   IL|    USA| ORD|
|   CLT|       Charlotte|   NC|    USA| CLT|
|   GSP|      Greenville|   SC|    USA| GSP|
|   BOS|          Boston|   MA|    USA| BOS|
|   COS|Colorado Springs|   CO|    USA| COS|
|   BNA|       Nashville|   TN|    USA| BNA|
|   ATL|         Atlanta|   GA|    USA| ATL|
|   SJC|        San Jose|   CA|    USA| SJC|
|   GJT|  Grand Junction|   CO|    USA| GJT|
|   AZO|       Kalamazoo|   MI|    USA| AZO|
|   ELP|         El Paso|   TX|    USA| ELP|
|   PDX|  

In [13]:
airtraffic. \
    select("Origin"). \
    distinct(). \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA"], "left"). \
    filter("IATA IS NULL"). \
    show()

+------+----+-----+-------+----+
|Origin|City|State|Country|IATA|
+------+----+-----+-------+----+
|   HDN|null| null|   null|null|
|   SJU|null| null|   null|null|
|   ITO|null| null|   null|null|
|   STT|null| null|   null|null|
|   CEC|null| null|   null|null|
|   CDC|null| null|   null|null|
|   PSG|null| null|   null|null|
|   ADK|null| null|   null|null|
|   KOA|null| null|   null|null|
|   OTZ|null| null|   null|null|
|   BQN|null| null|   null|null|
|   STX|null| null|   null|null|
|   PMD|null| null|   null|null|
|   PSE|null| null|   null|null|
|   SCC|null| null|   null|null|
|   SLE|null| null|   null|null|
+------+----+-----+-------+----+



In [14]:
airtraffic. \
    select("Origin"). \
    distinct(). \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA"], "left"). \
    filter("IATA IS NULL"). \
    count()

16